In [64]:
import numpy as np
from scipy import spatial
import array
import sqlite3
from fuzzywuzzy import process

In [25]:
conn = sqlite3.connect("/mnt/datassd/csci1951a-spoticry-data/acoustic.db")
c = conn.cursor()

In [26]:
c.execute('''
    SELECT DISTINCT song_id, vec
    FROM transformer
''')
data = c.fetchall()
conn.close()

In [35]:
conn = sqlite3.connect("/mnt/datassd/csci1951a-spoticry-data/spoticry.db")
c = conn.cursor()

In [36]:
c.execute('''
    SELECT DISTINCT song_id, artist, title
    FROM songs
''')
names = c.fetchall()
conn.close()

In [ ]:
#ids = dict(zip([d[0] for d in data], range(len(data))))
ids = [d[0] for d in data]
vecs = np.array([np.array(array.array("f", d[1])) for d in data])

In [ ]:
tree = spatial.KDTree(vecs)
id_to_name = dict([(d[0], d[1] + ' : ' + d[2]) for d in names if d[0]])
name_to_id = dict([(d[1] + ' : ' + d[2], d[0]) for d in names if d[0] if d[0] in ids])

In [54]:
def combine_songs(name1, name2, op):
    try:
        id1 = name_to_id[name1]
        id2 = name_to_id[name2]
        index1 = ids.index(id1)
        index2 = ids.index(id2)
    except:
        return None
    new = op(vecs[index1], vecs[index2])
    ind = tree.query(new)[1]
    return id_to_name[ids[ind]]
    

In [57]:
list(name_to_id.keys())

11163

In [56]:
out = combine_songs('Biffy Clyro : Love Has A Diameter', 'Luther Allison : K.T.',np.subtract)
print(out)

God Forbid : To The Fallen Hero


In [66]:
maybe = process.extract("Billy Joel : Vienna",list(name_to_id.keys()), limit=3)
print(maybe)

[('Billy Talent : Standing In The Rain', 86), ('Billy Ray Cyrus : Achy Breaky Heart', 86), ('Billy Fury : Halfway To Paradise', 86)]
